### tokenizer理解

In [ ]:
!huggingface-cli download --resume-download google-bert/bert-base-uncased --local-dir ../../model/bert-base-uncased --local-dir-use-symlinks False

In [1]:
from transformers import AutoModel,AutoTokenizer

d:\anaconda\envs\llm\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\anaconda\envs\llm\lib\site-packages\transformers\utils\hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [4]:
model_name = "../../model/bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to("cuda")
sentences = "this is a test sentence"

### model
1.模型详细结构
- model

2.模型整体结构
- model.config

3.模型参数
- model.num_parameters()
- eps 相当于$\epsilon$
- elementwise_affine 相当于bias
$$\mathrm{output}=\mathrm{weight}\cdot\frac{\mathrm{input}-\mu}{\sqrt{\sigma^2+\epsilon}}+\mathrm{bias}$$

- model.norm.weight :参考norm层详细的参数

### model(**tokens)
1.model(**tokens) 解析
- 输出得到last_hidden_state、pooler_output
- last_hidden_state维度:(batch_size,sequence_length,hidden_size)
- pooler_output维度:(batch_size,hidden_size)
- 可将两者用于下游任务，例如分类等任务
- pooler_output == model.pooler(last_hidden_state)

In [73]:
sentences = "i am so happy"
tokens = tokenizer(sentences, truncation = True,padding=True,max_length=256,return_tensors="pt")
tokens

{'input_ids': tensor([[ 101, 1045, 2572, 2061, 3407,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [14]:
tokenizer.tokenize(sentences)
tokenizer.encode(sentences)
tokenizer.decode([101, 2023, 2003, 1037, 8915, 13462, 6251, 102])
tokenizer.convert_ids_to_tokens([101, 2023, 2003, 1037, 8915, 13462, 6251, 102])

['[CLS]', 'this', 'is', 'a', 'te', '##set', 'sentence', '[SEP]']

In [19]:
tokenizer.convert_tokens_to_ids([ 'this', 'is', 'a', 'test',  'sentence'])

[2023, 2003, 1037, 3231, 6251]

In [21]:
tokenizer.convert_tokens_to_ids(sentences.split())

[2023, 2003, 1037, 3231, 6251]

In [26]:
tokenizer.special_tokens_map.values()
tokenizer.convert_tokens_to_ids(tokenizer.special_tokens_map.values())

[100, 102, 0, 101, 103]

In [54]:
import torch

In [ ]:
output = model(**tokens)
with torch.no_grad():
    output = model(**tokens)
    print(output)

In [59]:
output.pooler_output.shape

torch.Size([1, 768])

In [75]:
from transformers import BertForSequenceClassification

sequence_model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

with torch.no_grad():
    output = sequence_model(**tokens)
    softmax = output.logits.softmax(dim=-1)
    prediction = torch.argmax(softmax,dim=-1)

    print(output)
    print(softmax)
    print(prediction)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ../model/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


SequenceClassifierOutput(loss=None, logits=tensor([[-0.3806,  0.5056]]), hidden_states=None, attentions=None)
tensor([[0.2919, 0.7081]])
tensor([1])


In [88]:
sequence_model.config.label2id

{'LABEL_0': 0, 'LABEL_1': 1}

### tokenizer
1.查看分词情况:
- tokenizer.tokenize(sentences)

2.编码解码
- tokenizer.encode(sentences) = tokenizer.convert_tokens_to_ids()
- tokenizer.decode(senteencs) = tokenizer.convert_ids_to_tokens()

3.特殊编码
- tokenizer.special_tokens_map

4.词汇表
- tokenizer.vocab

5.输出解析
- ipute_ids:对输入进行vocal表的映射，即将自然语言转换成模型理解的语言
- attention_mask:可以理解为对词的关注
    - 如果有很多句子，len(attention_mask)等于最长句子的长度，目的是为了能够批次化输入，确保所有句子的输入长度都是一样的，跟padding=True相结合理解
- token_type_ids:用于区分不同的句子，len(token_type_ids)==len(attention_mask),一般用于句子对,相应的任务,可以理解为用于预测下一个句子是什么

In [14]:
sentences = ["this is a test sentence", 
             "i am so happy",
             "test sentence",
             "i am so sad,that so sad",]

In [19]:
tokens = tokenizer(sentences, truncation=True, padding=True, max_length=256, return_tensors="pt")


In [18]:
tokenizer.encode_plus(text=sentences[0],
                      text_pair=sentences[1],)  


{'input_ids': [101, 2023, 2003, 1037, 3231, 6251, 102, 1045, 2572, 2061, 3407, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}